In [ ]:
%pip install --upgrade --quiet transformers huggingface_hub
%pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks
%pip install --quiet --upgrade langchain langchain-text-splitters langchain-community
%pip install --quiet openai
%pip install --quiet pytesseract pillow pdf2image
import nltk
nltk.download('punkt_tab')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt_tab to C:\Users\Balaji
[nltk_data]     J\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import os
import re
import pytesseract
from pdf2image import convert_from_path

from pinecone import Pinecone, ServerlessSpec
from pinecone_text.sparse import BM25Encoder
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import PineconeHybridSearchRetriever

# ✅ Load config (handles .env locally or Secrets Manager on AWS)
from config import (
    PINECONE_API_KEY, 
    PINECONE_INDEX_NAME,
    HF_TOKEN
)

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = PINECONE_INDEX_NAME

# Create index if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"   # change if you created Pinecone index in another region
        )
    )

index = pc.Index(index_name)

# HuggingFace setup
os.environ["HF_TOKEN"] = HF_TOKEN
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
bm25_encoder = BM25Encoder().default()

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # change path on EC2 (Linux)

def clean_text(text: str) -> str:
    """Clean extracted text (remove extra spaces, line breaks)."""
    return " ".join(text.split())

def extract_text_using_ocr(pdf_path: str) -> str:
    """Extract text from all pages of a PDF file using OCR."""
    images = convert_from_path(
        pdf_path,
        poppler_path=r"C:\Users\Balaji J\OneDrive\Desktop\Release-25.07.0-0\poppler-25.07.0\Library\bin"  # change path on EC2 (Linux)
    )

    total_pages = len(images)
    all_text = []

    for page_num, image in enumerate(images):
        print(f"Extracting text from page {page_num + 1} of {total_pages}...")
        page_text = pytesseract.image_to_string(image)
        cleaned_page_text = clean_text(page_text)
        all_text.append(cleaned_page_text)

    return "\n".join(all_text)

In [ ]:
index = pc.Index(index_name)
index

Converting the Pdfs into text format


Processing file: humira.pdf...
Extracting text from page 1 of 111...
Extracting text from page 2 of 111...
Extracting text from page 3 of 111...
Extracting text from page 4 of 111...
Extracting text from page 5 of 111...
Extracting text from page 6 of 111...
Extracting text from page 7 of 111...
Extracting text from page 8 of 111...
Extracting text from page 9 of 111...
Extracting text from page 10 of 111...
Extracting text from page 11 of 111...
Extracting text from page 12 of 111...
Extracting text from page 13 of 111...
Extracting text from page 14 of 111...
Extracting text from page 15 of 111...
Extracting text from page 16 of 111...
Extracting text from page 17 of 111...
Extracting text from page 18 of 111...
Extracting text from page 19 of 111...
Extracting text from page 20 of 111...
Extracting text from page 21 of 111...
Extracting text from page 22 of 111...
Extracting text from page 23 of 111...
Extracting text from page 24 of 111...
Extracting text from page 25 of 111...
Ex

In [60]:
import os

all_text = ""
for file_name in os.listdir(output_text_folder):
    if file_name.endswith(".txt"):
        file_path = os.path.join(output_text_folder, file_name)
        with open(file_path, "r", encoding="utf-8") as file:
            all_text += file.read()
chunk_size = 300
sentences = [all_text[i:i + chunk_size] for i in range(0, len(all_text), chunk_size)]

In [61]:
bm25_encoder.fit(sentences)

bm25_encoder.dump("rag-veda.json")

100%|██████████| 1976/1976 [00:01<00:00, 1420.20it/s]


In [62]:
bm25_encoder = BM25Encoder().load("rag-veda.json")

In [63]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [64]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000024CA32234A0>, index=<pinecone.db_data.index.Index object at 0x0000024C8B12E870>)

In [65]:
response = retriever.add_texts(sentences)

100%|██████████| 62/62 [01:23<00:00,  1.35s/it]
